In [1]:
import pandas as pd
import os


In [2]:
#import the csv file
file_path = os.path.join("original_data/daily_counts_of_cases_hospitalizations_and_deaths.csv")
file_path2 = os.path.join("original_data/case_by_boroughs.csv")

#open the csv file as read
df = pd.read_csv(file_path)

#convert the dtype of the date column from object to datetime
df['DATE_OF_INTEREST'] = pd.to_datetime(df['DATE_OF_INTEREST'])

#rename the columns
df.columns = ["Date", "Case Count", "Hospitalized Count", "Death Count"]

#select the date of interest
covid_df = df.loc[1:122]
#display
covid_df.head()

,Date,Case Count,Hospitalized Count,Death Count
1,2020-03-01,0,4,0
2,2020-03-02,0,21,0
3,2020-03-03,2,20,0
4,2020-03-04,5,22,0
5,2020-03-05,3,20,0


In [3]:
#open the csv file as read
df2 = pd.read_csv(file_path2)

#select the date of interest
covid_boroughs_df = df2.loc[1:122]

#select the case count
covid_boroughs_df = covid_boroughs_df[["DATE_OF_INTEREST", "BK_CASE_COUNT", "BX_CASE_COUNT","MN_CASE_COUNT", "QN_CASE_COUNT", "SI_CASE_COUNT"]]

#rename the column
covid_boroughs_df.columns = ['Date', 'Brooklyn', "Bronx", "Manhattan", "Queens", "Staten Island"]

#display
covid_boroughs_df.head()

,Date,Brooklyn,Bronx,Manhattan,Queens,Staten Island
1,03/01/2020,0,0,0,0,0
2,03/02/2020,0,0,0,0,0
3,03/03/2020,0,1,0,1,0
4,03/04/2020,1,0,2,2,0
5,03/05/2020,3,0,0,0,0


In [ ]:
#save the dataframe as csv files
covid_df.to_csv("covid_cases.csv")
covid_boroughs_df.to_csv("covid_boroughs.csv")

In [4]:
#import csv files of transportations
df = pd.DataFrame()
for month in ['03','04','05','06']:
    df = df.append(pd.read_csv("original_data/yellow_tripdata_2019-"+month+".csv" , low_memory = False), ignore_index=True)
    
#rename the location column to match the taxi zone
df.rename(columns={'PULocationID':'LocationID'}, inplace=True)
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'], format='%Y-%m-%d %H:%M:%S')

#select the columns
df2 = df[['LocationID', 'tpep_pickup_datetime']]

#change the datetime column to date 
df2['tpep_pickup_datetime'] = df2['tpep_pickup_datetime'].dt.date

#import the taxi zone csv file
zone_lookup = pd.read_csv("original_data/taxi_zone_lookup.csv") # this dataset have a map between the LocationID and Borough

#merge with the taxi csv file to locate the boroughs
yellow_taxi_2019_boroughs = df2.merge(zone_lookup[['LocationID', 'Borough']], how='inner' ,on='LocationID').fillna(0)
yellow_taxi_2019_boroughs = yellow_taxi_2019_boroughs.drop(['LocationID'], axis=1)
yellow_taxi_2019_boroughs.head()

<ipython-input-4-a458cd4afe55>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['tpep_pickup_datetime'] = df2['tpep_pickup_datetime'].dt.date


,tpep_pickup_datetime,Borough
0,2019-03-01,Queens
1,2019-03-01,Queens
2,2019-03-01,Queens
3,2019-03-01,Queens
4,2019-03-01,Queens


In [5]:
#create a dataframe with total usage count
yellow_2019= pd.DataFrame(df2.tpep_pickup_datetime.value_counts().reset_index().values, columns=["Date", "Yellow_Taxi_2019"])
yellow_2019 = yellow_2019.sort_values(by="Date")

#display the data
yellow_2019.head()

,Date,Yellow_Taxi_2019
161,2001-01-01,2
159,2002-12-31,2
152,2008-08-08,3
124,2008-12-31,52
123,2009-01-01,148


In [ ]:
#save the dataframe as csv file
yellow_2019_borough.to_csv("yellow_2019_borough.csv")
yellow_2019.to_csv("yellow_2019.csv")